# UNK token을 포함한 문장에는 무엇이 있는가 확인하는 ipynb

## 1. 기초 함수 세팅

In [1]:
import json

import pandas as pd
from matplotlib import pyplot as plt

from tqdm import tqdm
from transformers import AutoTokenizer

None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


In [2]:
# unk sentence to csv

from collections import Counter, OrderedDict

train_path = './data/train.csv'
dev_path = './data/dev.csv'

train_data = pd.read_csv(train_path)
dev_data = pd.read_csv(dev_path)

tokenizer = AutoTokenizer.from_pretrained('klue/roberta-small')

# Function Settings
def get_num_tokens(df):
    sentence1_len, sentence2_len = [], []
    sentence1_unk, sentence2_unk = [], []
    
    for i, item in df.iterrows():
        sentence1 = tokenizer(item['sentence_1'])['input_ids']
        sentence2 = tokenizer(item['sentence_2'])['input_ids']

        sentence1_len.append(len(sentence1))
        sentence2_len.append(len(sentence2))

        sentence1_unk.append(sentence1.count(tokenizer.unk_token_id))
        sentence2_unk.append(sentence2.count(tokenizer.unk_token_id))

    return sentence1_len, sentence2_len, sentence1_unk, sentence2_unk
    # return pd.DataFrame({'number of tokens':sentence1_len, 'label score':df.label.values.tolist()})


# 1. 전체 df에 대해 score 5단계로 분류, 열 추가
train_data_scored = train_data.copy(deep=True)
score_integer = []

for i, item in train_data_scored.iterrows():
    label_value = int(item['label'])
    if   label_value == 0:  col = 0
    elif label_value < 2.0: col = 1
    elif label_value < 3.0: col = 2
    elif label_value < 4.0: col = 3
    elif label_value < 5.0: col = 4
    else:                   col = 5
        
    score_integer.append(col)
train_data_scored['score_class'] = score_integer

# 2. sentence 별 토큰 개수 넣기
s1_len, s2_len, s1_unk, s2_unk = get_num_tokens(train_data_scored)

train_data_scored['s1_num_tokens'] = s1_len
train_data_scored['s2_num_tokens'] = s2_len
train_data_scored['s1_num_unk'] = s1_unk
train_data_scored['s2_num_unk'] = s2_unk

# 2. unk token 문장 확인 후 CSV 저장

In [3]:
# unk sentence to csv

unk1_token_sentence = []
unk2_token_sentence = []
unk1_count, unk2_count = [], []

source_list = sorted(train_data['source'].unique())

for source_name in source_list:
    g = train_data_scored.groupby(['source']).get_group(source_name)
    u1_sentence = g[g['s1_num_unk'] >=1]['sentence_1'].values.tolist()
    u2_sentence = g[g['s2_num_unk'] >=1]['sentence_2'].values.tolist()

    unk1_token_sentence.extend(u1_sentence)
    unk2_token_sentence.extend(u2_sentence)
    unk1_count.extend(g[g['s1_num_unk'] >=1]['s1_num_unk'].values.tolist())
    unk2_count.extend(g[g['s2_num_unk'] >=1]['s2_num_unk'].values.tolist())

_t_num = len(unk1_token_sentence) + len(unk2_token_sentence)
print(f"<UNK> 토큰을 포함하고 있는 문장의 전체 개수는 {_t_num} 개 입니다.")

max_len = max(len(unk1_token_sentence), len(unk2_token_sentence))
unk1_token_sentence += [''] * (max_len - len(unk1_token_sentence))
unk1_count += [''] * (max_len - len(unk1_count))
unk2_token_sentence += [''] * (max_len - len(unk2_token_sentence))
unk2_count += [''] * (max_len - len(unk2_count))

unk_pd = pd.DataFrame({'unk1_sentences': unk1_token_sentence, 
                       'unk1_count': unk1_count,
                       'unk2_sentences': unk2_token_sentence,
                       'unk2_count': unk2_count,
                       })
unk_pd.to_csv('./unk_setences_train.csv')
print(f"해당 문장들을 csv파일로 저장하였습니다. 파일 이름: unk_setences_train.csv")

<UNK> 토큰을 포함하고 있는 문장의 전체 개수는 453 개 입니다.
해당 문장들을 csv파일로 저장하였습니다. 파일 이름: unk_setences_train.csv


# 3. 어떤 단어가 UNK로 인식되었는지 확인

In [5]:
# train data에서 어떤 단어가 <UNK>로 인식되는 것인지 확인해보자.

# 재밌는 것: encode = tokenizer(text) 의 결과는 transformer에 속하는 타입으로, 내장 함수가 몇 가지 있다. ex. token_to_char ...
#print(type(encoded))

# 0. unk_pd가 저장되었다고 가정.

# 0-1. setting
import copy
# 1. 문장에 대한 반복

def find_unk_tokens(ex_unk_sentences):
    unk_tokens = []
    count = 0
    
    for unk_sentence in ex_unk_sentences:
        _unk = []
        # 2. encode and decode
        encoded = tokenizer(unk_sentence)                                
        decoded = tokenizer.convert_ids_to_tokens(encoded['input_ids'])     # 각 token id를 리스트로 리턴

        # 3. 몇 번째가 unk인지 확인. 인덱스 모두 저장
        unk_indexes = [i for i, token in enumerate(decoded) if token == tokenizer.unk_token]
        count += len(unk_indexes)

        # 4. input에서 어떤 단어가 unk였는지 확인
        for index in unk_indexes:
            char_index = encoded.token_to_chars(index)
            original_token = unk_sentence[char_index.start:char_index.end]  # char_index 는 CharSpan(start=15, end=19) 형태로 리턴되더랍니다... 신기!
            
            #print(f"<UNK> token: {original_token}")
            _unk.append(original_token)
        
        if _unk:
            unk_tokens.append(_unk)
            
    return unk_tokens, count

unk1_sentences, s1_count = find_unk_tokens(unk_pd['unk1_sentences'].values.tolist())
unk2_sentences, s2_count  = find_unk_tokens(unk_pd['unk2_sentences'].values.tolist())
unk_tokens = copy.deepcopy(unk1_sentences)
unk_tokens.extend(unk2_sentences)
print(f"unk 로 인식된 토큰들 : {unk_tokens}\nunk로 인식된 토큰의 총 개수는 {s1_count + s2_count} 개 입니다.")

# 5. 기존 unk_pd(sentence가 저장된 csv) 에 저장
# 5-1. 길이 맞추기
max_len = max(len(unk1_sentences), len(unk1_sentences))
unk2_sentences += [''] * (max_len - len(unk2_sentences))
unk_pd['unk1_token'] = unk1_sentences
unk_pd['unk2_token'] = unk2_sentences

unk_pd = unk_pd[['unk1_sentences', 'unk1_count', 'unk1_token', 'unk2_sentences', 'unk2_count', 'unk2_token']]
unk_pd.head(5)

# # 5. csv 로 저장
# unk_tokens_pd = pd.DataFrame({'unk_token': unk_tokens})
# unk_tokens_pd.to_csv("./unk_tokens_train.csv")
unk_pd.to_csv("./unk_tokens_train.csv")

print(f"unk 로 인식된 토큰 리스트를 저장하였습니다. 파일 이름: unk_tokens_train.csv")

unk 로 인식된 토큰들 : [['오마이가뜨지져스크롸이스트휏'], ['봣는데'], ['봣는데'], ['퀼리티를'], ['됫는데'], ['줸쟝'], ['스타뎀의'], ['갖췃다'], ['스타뎀이나'], ['바르뎀의'], ['됏엇지만'], ['안봣다ㅡㅡ'], ['콱'], ['펭귄이'], ['멎진'], ['끅끅끅'], ['봣습니다'], ['꺅'], ['꿎꿎하게'], ['만드셧어'], ['사랑이뤄지길바럤는데'], ['괸찮다'], ['봣다'], ['맬'], ['믕지가'], ['훠어얼씬'], ['웩'], ['구가의서가훨씬낫다ㅉ'], ['두번봣는데'], ['퀵을'], ['번쨰에서'], ['티비로봣는데잼서요'], ['어딨으랴'], ['괞찮았다'], ['앳킨스'], ['수고하셧습니다'], ['ㅑㅋ', 'ㅑ', 'ㅑ'], ['뽈노'], ['영화네욥'], ['영호ㅓ'], ['반담횽'], ['겜'], ['편집됬다는게', '빂츠갔더니', '뙇'], ['뗀적이'], ['집중잘안됏던'], ['멋이쑈어여'], ['펭귄맨'], ['앜ㅋㅋ웃는거야', '우는거얔ㅋㅋㅋ'], ['매젬게'], ['사람이저렇게홱홱변할수가있나어처구니없다'], ['헿'], ['힛걸에의한', '힛걸을위한', '힛걸에'], ['ㅉㅉ'], ['ㅤㅅㅞㅅ'], ['검색해봤는뎁ㅋㅋ'], ['홧팅'], ['숀'], ['소꿉놀이하는'], ['되고싶어욯ㅎㅎ'], ['샹ㅂ나ㅓ'], ['틀어주는거봣는대'], ['왤케'], ['마지막횐줄모르고봣는데'], ['쑈를', '쑈', '쑈', '쑈'], ['ㅉㅉ볼줄모르면'], ['ㅉㅉ'], ['맞죵'], ['찝찝하고'], ['왤케'], ['좠는데'], ['액숀'], ['반감됬다'], ['꼬꼬마들ㅉㅉ'], ['ㅉㅉㄱ'], ['얍'], ['완전완줜'], ['됬냐'], ['조니뎁'], ['로완앳킨슨'], ['화씨과함꼐'], ['봣는데'], ['뿅'], ['호호홐'], ['왤캐'], ['ㅉㅉ'], ['웤을'], ['무조껀'], ['제ㅔㅔ발'], ['역시일본것칻더니일본거내'], ['어릴떄'], ['잼있었늣데'

# 4. UNK를 포함한 문장들을 spelling check 진행
- git clone 하고,
- pyhanspell -> issue 들어가서 바꾸라는대로 바꾸고 재설치.

In [6]:
!git clone https://github.com/ssut/py-hanspell.git

# 이후 https://github.com/ssut/py-hanspell/issues/31 에 따라 파일 내용 수정 필요

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
'py-hanspell'에 복제합니다...
remote: Enumerating objects: 101, done.
remote: Counting objects: 100% (20/20), done.
remote: Compressing objects: 100% (17/17), done.
remote: Total 101 (delta 5), reused 10 (delta 3), pack-reused 81
오브젝트를 받는 중: 100% (101/101), 25.27 KiB | 8.42 MiB/s, 완료.
델타를 알아내는 중: 100% (42/42), 완료.


In [8]:
%cd ./py-hanspell
!python ./setup.py install

%cd ../

/Users/ilewis/git_dir/level1_semantictextsimilarity-nlp-14/JH/data_analysis/py-hanspell
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
/Users/ilewis/opt/anaconda3/lib/python3.9/site-packages/setuptools/dist.py:771: UserWarning: Usage of dash-separated 'description-file' will not be supported in future versions. Please use the underscore name 'description_file' instead
  warnings.warn(
running install
/Users/ilewis/opt/anaconda3/lib/python3.9/site-packages/setuptools/command/install.py:34: SetuptoolsDeprecationWarning: setup.py install is deprecated. Use build and pip and other standards-based tools.
  warnings.warn(
/Users/ilewis/opt/anaconda3/lib/python3.9/site-packages/setuptools/command/easy_install.py:144: Eas

In [ ]:
# 맞춤법 교정 후 복원한 문장을 옆에 붙일 예정입니다. 위 셀들로부터 이어서, unk_pd 가 불러와져있다고 가정합니다.

# 0. settings
import sys
sys.path.append('./py-hanspell')
from tqdm import tqdm
from hanspell import spell_checker

# 1. 스펠링 체크를 진행할 문장 세트 준비
u1_s = unk_pd['unk1_sentences'].values.tolist()
u2_s = unk_pd['unk2_sentences'].values.tolist()
u2_s = [item for item in u2_s if item]

# 2. 함수 선언
def check(data):
    changed, label = [], []
    
    for sentence in tqdm(data):
        # 2-1. 스펠링 체크
        result = spell_checker.check(sentence).as_dict()
        checked = result['checked']
        
        # 2-2. 결과. 문장이 전~혀 바뀌지 않는 경우에만 unchanged라고 기록되게 함. 단순히 띄어쓰기만 들어가도 changed 로 인식됨.
        changed.append(checked)
        label += ["changed" if sentence!=checked else "unchanged"]

    return changed, label

# 3. pd 저장
changed_1, label_1 = check(u1_s)
changed_2, label_2 = check(u2_s)

# 3-1. 길이가 서로 다른 리스트를 '열' 로 붙여넣으려고 하니까, 길이가 안 맞으면 안 되서, 임시 방편으로 넣어뒀습니다. 이거때문에 좀 오히려 복잡해져서 수정예정입니다.
changed_2 += [''] * (len(changed_1) - len(changed_2))
label_2 += [''] * (len(label_1) - len(label_2))

unk_pd['unk1_sentences_checked'] = changed_1
unk_pd['unk1_checked_label'] = label_1


unk_pd['unk2_sentences_checked'] = changed_2
unk_pd['unk2_checked_label'] = label_2

unk_pd = unk_pd[['unk1_sentences', 'unk1_count', 'unk1_token', 'unk1_sentences_checked', 'unk1_checked_label', 
                 'unk2_sentences', 'unk2_count', 'unk2_token', 'unk2_sentences_checked', 'unk2_checked_label']]

unk_pd.head(5)